[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/mnist_dataset_using_normal_nn_1_06_2018/blob/master/mnist_dataset_using_normal_nn.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("drive/mnist dataset using normal nn", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting drive/mnist dataset using normal nn/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting drive/mnist dataset using normal nn/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting drive/mnist dataset using normal nn/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting drive/mnist dataset using normal nn/t10k-labels-i

In [5]:
os.chdir('/content')
!git clone https://github.com/mixuala/colab_utils
import colab_utils.tboard
ROOT = %pwd
print(ROOT)
LOG_DIR = os.path.join(ROOT, 'gaurav')
print(LOG_DIR)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

Cloning into 'colab_utils'...
remote: Counting objects: 235, done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 235 (delta 6), reused 11 (delta 2), pack-reused 220
Receiving objects: 100% (235/235), 74.58 KiB | 942.00 KiB/s, done.
Resolving deltas: 100% (91/91), done.
/content
/content/gaurav
calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= http://e7879349.ngrok.io


'http://e7879349.ngrok.io'

In [0]:
no_of_layers = tf.constant(3, name = "no_of_layers")
no_of_nodes_l1 = tf.constant(500, name = "no_of_nodes_l1")
no_of_nodes_l2 = tf.constant(500, name = "no_of_nodes_l2")
no_of_nodes_in_output_layer = tf.constant(10, name = "no_of_nodes_in_output_layer")
batch_size = 128
hm_epochs = 10

In [0]:
x = tf.placeholder(dtype = tf.float32, shape = [None, None], name = "input_labels")
y = tf.placeholder(dtype = tf.float32, shape = [None, None], name = "output_labels")

In [0]:
def neural_network_model(data):
  
  with tf.name_scope('neural_network'):
  
    hidden_layer1 = {"weights_l1" : tf.Variable(tf.random_normal([784, no_of_nodes_l1])),
                     "biases_l1" : tf.Variable(tf.random_normal([1, no_of_nodes_l1]))}
    hidden_layer2 = {"weights_l2" : tf.Variable(tf.random_normal([no_of_nodes_l1, no_of_nodes_l2])),
                     "biases_l2" : tf.Variable(tf.random_normal([1, no_of_nodes_l2]))}
    output_layer = {"weights_ol": tf.Variable(tf.random_normal([no_of_nodes_l2, no_of_nodes_in_output_layer])), 
                   "biases_ol": tf.Variable(tf.random_normal([1, no_of_nodes_in_output_layer]))}

    z1 = tf.add(tf.matmul(data, hidden_layer1["weights_l1"]), hidden_layer1["biases_l1"])
    a1 = tf.nn.relu(z1)
    z2 = tf.add(tf.matmul(a1, hidden_layer2["weights_l2"]), hidden_layer2["biases_l2"])
    a2 = tf.nn.relu(z2)
  
    output = tf.add(tf.matmul(a2, output_layer["weights_ol"]), output_layer["biases_ol"])

    return output

In [0]:
def train_neural_network(x):

  prediction = neural_network_model(x)
  
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
  summary = tf.summary.scalar('total_cost_of_batch', cost)
  
  with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  merged = tf.summary.merge_all()
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./gaurav',sess.graph)
    k = 0
    for i in range(hm_epochs):
      epoch_loss = 0
      for j in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        o, c, m = sess.run([optimizer, cost, merged], feed_dict = {x : epoch_x, y : epoch_y})
        epoch_loss += c
        train_writer.add_summary(m, k)
        k += 1
      print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)
      
      
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
  
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
  
    print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels})) 

In [48]:
train_neural_network(x)

epoch 0 completed out of  10 loss  113543.1298904419
epoch 1 completed out of  10 loss  29520.10003376007
epoch 2 completed out of  10 loss  17828.107043981552
epoch 3 completed out of  10 loss  11445.93170452118
epoch 4 completed out of  10 loss  7563.912250354886
epoch 5 completed out of  10 loss  5491.495393703692
epoch 6 completed out of  10 loss  3786.446009475723
epoch 7 completed out of  10 loss  2804.525605773786
epoch 8 completed out of  10 loss  2097.379685644888
epoch 9 completed out of  10 loss  1350.1484947205056
Accuracy: 0.9503
